In [38]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import datetime
import boto3
import psycopg2
import configparser

#### Iniciación de Variables

In [39]:
cantidad_clientes = np.random.randint(500, 1000) #cantidad de clientes a crear
rdsIdentifier = 'proyecto' #nombre de la instancia
fake = Faker() #inicialización para creación de data random

#### Cargamos archivo de configuraciones

In [40]:
config = configparser.ConfigParser()
config.read('proy_escec.cfg')

['proy_escec.cfg']

### Creamos Instancia de RDS 

In [43]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')

#### Verificamos Instancias de RDS disponibles

In [44]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['dw', 'proyecto']


#### Creación de Servicio RDS

In [45]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


##### Recordemos Esperar unos minutos para consultar la informaicón de la instancia.

##### Obtenemos URL del Host

In [46]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

proyecto.cf32hjwhygka.us-east-1.rds.amazonaws.com


##### Conexión a Base de Datos desde Python

In [47]:
import Proy_sql_queries

try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(Proy_sql_queries.DDL_QUERY)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Base de Datos Creada Exitosamente


##### Insertamos Datos en tablas

In [48]:
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

Insertamos datos en Lenguaje

In [49]:
data_lenguaje = [
     {'id_lenguaje': 10, 'lenguaje': 'español'}, 
     {'id_lenguaje': 11, 'lenguaje': 'ingles'}, 
     {'id_lenguaje': 12, 'lenguaje': 'portugues'}
]

#insertamos data en tabla lenguaje 
insertDataToSQL(data_lenguaje, 'lenguaje')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "lenguaje_pkey"
DETAIL:  Key (id_lenguaje)=(10) already exists.

[SQL: INSERT INTO lenguaje (id_lenguaje, lenguaje) VALUES (%(id_lenguaje)s, %(lenguaje)s)]
[parameters: ({'id_lenguaje': 10, 'lenguaje': 'español'}, {'id_lenguaje': 11, 'lenguaje': 'ingles'}, {'id_lenguaje': 12, 'lenguaje': 'portugues'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


#### Insertamos Data para Genero de libro

In [50]:
data_genero = [
     {'id_genero': 20, 'genero': 'suspenso'}, 
     {'id_genero': 21, 'genero': 'terror'}, 
     {'id_genero': 22, 'genero': 'superacion'}, 
     {'id_genero': 23, 'genero': 'novela'}
]

insertDataToSQL(data_genero, 'genero')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "genero_pkey"
DETAIL:  Key (id_genero)=(20) already exists.

[SQL: INSERT INTO genero (id_genero, genero) VALUES (%(id_genero)s, %(genero)s)]
[parameters: ({'id_genero': 20, 'genero': 'suspenso'}, {'id_genero': 21, 'genero': 'terror'}, {'id_genero': 22, 'genero': 'superacion'}, {'id_genero': 23, 'genero': 'novela'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


##### Insertamos Data para Autor

In [51]:
data_autor = [
     {'id_autor': 30, 'autor': 'Juan Soto'}, 
     {'id_autor': 31, 'autor': 'Susana Rojas'}, 
     {'id_autor': 32, 'autor': 'Pedro Fernandez'}, 
     {'id_autor': 33, 'autor': 'Rocio Chacon'}, 
     {'id_autor': 34, 'autor': 'Julio Quezada'}
]

insertDataToSQL(data_autor, 'autor')


(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "autor_pkey"
DETAIL:  Key (id_autor)=(30) already exists.

[SQL: INSERT INTO autor (id_autor, autor) VALUES (%(id_autor)s, %(autor)s)]
[parameters: ({'id_autor': 30, 'autor': 'Juan Soto'}, {'id_autor': 31, 'autor': 'Susana Rojas'}, {'id_autor': 32, 'autor': 'Pedro Fernandez'}, {'id_autor': 33, 'autor': 'Rocio Chacon'}, {'id_autor': 34, 'autor': 'Julio Quezada'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


##### Insertamos Data para Editorial

In [52]:
data_editorial = [
     {'id_editorial': 40, 'editorial': 'Libros Rotos'}, 
     {'id_editorial': 41, 'editorial': 'Reflejos'},
     {'id_editorial': 42, 'editorial': 'Profesional'},
     {'id_editorial': 43, 'editorial': 'Educativa'}
]

insertDataToSQL(data_editorial, 'editorial')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "editorial_pkey"
DETAIL:  Key (id_editorial)=(40) already exists.

[SQL: INSERT INTO editorial (id_editorial, editorial) VALUES (%(id_editorial)s, %(editorial)s)]
[parameters: ({'id_editorial': 40, 'editorial': 'Libros Rotos'}, {'id_editorial': 41, 'editorial': 'Reflejos'}, {'id_editorial': 42, 'editorial': 'Profesional'}, {'id_editorial': 43, 'editorial': 'Educativa'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


##### Insertamos Data para Sucursales

In [53]:
data_sucursal = [
    {'id_sucursal': 50, 'sucursal': 'zona 10'},
    {'id_sucursal': 51, 'sucursal': 'Centro civico'},
    {'id_sucursal': 52, 'sucursal': 'Peten'},
    {'id_sucursal': 53, 'sucursal': 'Quetzaltenango'}
]
insertDataToSQL(data_sucursal, 'sucursal')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "sucursal_pkey"
DETAIL:  Key (id_sucursal)=(50) already exists.

[SQL: INSERT INTO sucursal (id_sucursal, sucursal) VALUES (%(id_sucursal)s, %(sucursal)s)]
[parameters: ({'id_sucursal': 50, 'sucursal': 'zona 10'}, {'id_sucursal': 51, 'sucursal': 'Centro civico'}, {'id_sucursal': 52, 'sucursal': 'Peten'}, {'id_sucursal': 53, 'sucursal': 'Quetzaltenango'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


##### Insertamos Data para Clientes

In [54]:
data_cliente = []

for cliente in range(cantidad_clientes):
    clientProfile = fake.profile()
    nuevo_cliente = {
        'id_cliente': random.randint(10000000, 99999999),
        'nombre': clientProfile['name'].split(' ')[0],
        'apellido': clientProfile['name'].split(' ')[1],
        'correo': clientProfile['mail'],
        'telefono': clientProfile['ssn'],
    }

    data_cliente.append(nuevo_cliente)

insertDataToSQL(data_cliente, 'cliente')

Se han insertado 812 nuevos registros.


##### Insertamos Data de Libros

In [55]:
cantidad_libros = np.random.randint(50,150)
data_libros = []

for book in range (cantidad_libros):
    titulolibro = fake.profile()
    nuevo_libro = {
        'id_libro': random.randint(10000,99999),
        'titulo' : titulolibro ['name'],
        'id_lenguaje' : random.sample (data_lenguaje, 1)[0]['id_lenguaje'],
        'id_genero' : random.sample (data_genero, 1)[0]['id_genero'],
        'id_editorial' : random.sample (data_editorial, 1)[0]['id_editorial'],
        'id_autor' : random.sample (data_autor, 1)[0]['id_autor']
     }
    
    data_libros.append(nuevo_libro)
insertDataToSQL(data_libros, 'libros')


Se han insertado 93 nuevos registros.


##### Insertamos Data de Orden

In [58]:
cantidad_ordenes = np.random.randint(100, 200)
data_orden = []

for orden in range(cantidad_ordenes):
    nueva_orden = {
            'id_orden': orden,
            'fecha_hora': fake.date_time_this_year(),
            
            'id_libro': random.sample(data_libros, 1)[0]['id_libro'],
            'id_sucursal':  random.sample(data_sucursal, 1)[0]['id_sucursal'],
            'id_cliente': random.sample(data_cliente, 1)[0]['id_cliente']
        }
    
    data_orden.append(nueva_orden)
   
insertDataToSQL(data_orden, 'orden')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "orden_pkey"
DETAIL:  Key (id_orden)=(0) already exists.

[SQL: INSERT INTO orden (id_orden, fecha_hora, id_libro, id_sucursal, id_cliente) VALUES (%(id_orden)s, %(fecha_hora)s, %(id_libro)s, %(id_sucursal)s, %(id_cliente)s)]
[parameters: ({'id_orden': 0, 'fecha_hora': datetime.datetime(2023, 2, 20, 15, 59, 10), 'id_libro': 90168, 'id_sucursal': 50, 'id_cliente': 60237074}, {'id_orden': 1, 'fecha_hora': datetime.datetime(2023, 3, 18, 2, 2, 38), 'id_libro': 94874, 'id_sucursal': 53, 'id_cliente': 28021022}, {'id_orden': 2, 'fecha_hora': datetime.datetime(2023, 3, 10, 20, 33, 10), 'id_libro': 94039, 'id_sucursal': 50, 'id_cliente': 23154873}, {'id_orden': 3, 'fecha_hora': datetime.datetime(2023, 2, 13, 16, 36, 56), 'id_libro': 28012, 'id_sucursal': 50, 'id_cliente': 78942456}, {'id_orden': 4, 'fecha_hora': datetime.datetime(2023, 2, 8, 1, 20, 45), 'id_libro': 50794, 'id_sucursal': 50, 'id_cliente': 31217532}

In [57]:
fake.profile()

{'job': 'Financial trader',
 'company': 'Watts PLC',
 'ssn': '292-33-6529',
 'residence': '24786 Jenkins Radial\nLake Ryan, AK 29057',
 'current_location': (Decimal('35.5215225'), Decimal('-153.549317')),
 'blood_group': 'A-',
 'website': ['http://fisher.com/'],
 'username': 'michellewolfe',
 'name': 'Lori Crawford',
 'sex': 'F',
 'address': '113 Trevor Trail\nPort Matthew, NM 75854',
 'mail': 'owashington@gmail.com',
 'birthdate': datetime.date(1940, 9, 22)}